<a href="https://colab.research.google.com/github/adisakshya/playground/blob/remote-playground/remote/playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# System Specifications

Describe the Google Compute Engine backend powering this google-colab session.

In [ ]:
# Linux kernel version
!uname -a

In [ ]:
# OS name and version
!cat /etc/lsb-release

In [ ]:
# CPU architecture information
!lscpu

In [ ]:
# Check disk space used by filesystem 
!df -h

In [ ]:
# Check amount of free and used memory (both physical and swap)
# on the system as well as the shared memory and buffers used by the kernel
!cat /proc/meminfo

# Who am I?

Get the user name associated with the current effective user ID or the currently logged user.

In [ ]:
# Who am I?
!whoami

# Environment Setup

Create your collection of procedures and tools for developing, testing and debugging an application or program.

In [ ]:
# Update the package lists for upgrades
!apt update

In [ ]:
%%shell

# Install required tools and utilities
apt install \
    build-essential \
    httpie \
    software-properties-common

# Update npm
npm install -g npm

# Install gtop - https://github.com/aksakalli/gtop
npm install -g gtop

In [ ]:
# Install code-server - https://github.com/cdr/code-server
!curl -fsSL https://code-server.dev/install.sh | sh

In [ ]:
%%shell

# Install localtunnel - https://github.com/localtunnel/localtunnel
npm install -g localtunnel

# Install ngrok - https://ngrok.com
wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
unzip ngrok-stable-linux-amd64.zip
mv ./ngrok /usr/bin/ngrok

In [ ]:
# Create directory to contain log files
!mkdir logs

# Activate playground


Start code-server and increase its power through extensions.

In [ ]:
# Start code-server - https://github.com/cdr/code-server
# redirect standard output and standard error to different files
# logs can be found in the logs/ directory
!nohup code-server --port 8000 > logs/code_server.out 2> logs/code_server.err &

Extensions let you add languages, debuggers and tools to your installation to support your development workflow.

In [ ]:
%%shell

# Install required extensions
extensions=(
    'github.github-vscode-theme'
    'grapecity.gc-excelviewer'
    'ms-python.python'
    'ms-toolsai.jupyter'
    'ms-vscode.cpptools'
    'ritwickdey.liveserver'
    'vscode-icons-team.vscode-icons'
)
for extension in ${extensions[@]}; 
    do code-server --install-extension ${extension};
done

code-server will generate a password for you by default which can be found at ```~/.config/code-server/config.yaml```.

In [ ]:
%%shell

# Show code-server logs
cat logs/code_server.out
echo ---------------------
# Show code-server config
cat ~/.config/code-server/config.yaml

# Accessing playground

code-server by default runs on localhost and needs to be securely exposed to the internet to be able to access it from a web browser. 

localtunnel creates a tunnel to the specified local port offering secure https for all tunnels.

In [ ]:
# Start localtunnel - https://github.com/localtunnel/localtunnel
# redirect standard output and standard error to different files
# logs can be found in the logs/ directory
!nohup lt --port 8000 > logs/cdr_tunnel.out 2> logs/cdr_tunnel.err &

1. The above command will connect to the tunnel server, setup the tunnel, and tell you what url to use for your testing.
2. **This tunnel-url is stored in ```logs/cdr_tunnel.out```**.
3. This url will remain active for the duration of your session; so feel free to share it with others for peer programming.

In [ ]:
# Show localtunnel logs => tunnel-url
!cat logs/cdr_tunnel.out